## Tarea

Importa las librerías necesarias para resolver los siguientes ejercicios (no necesitaras ninguna que no hayamos utilizado nosotros)

In [ ]:
import geopandas as gpd 
import pandas as pd
import matplotlib.pyplot as plt
import re
import matplotlib.patches as mpatches
from shapely.ops import nearest_points

#### 1.
Grafica la división estatal de México (usando el shapefile que usamos en la clase) en diferentes tonos de algún color que elijas. Pintalos según su área. Los estados más oscuros siendo los más grandes

In [2]:
# Cargar el Shapefile
gdf = gpd.read_file("/usr/src/app/Estados/conjunto_de_datos/00ent.shp")

# Calcular el área de cada estado y añadirlo como una nueva columna
gdf['area'] = gdf['geometry'].area

# Normalizar el área para que los colores varíen entre un rango específico
gdf['area_norm'] = (gdf['area'] - gdf['area'].min()) / (gdf['area'].max() - gdf['area'].min())

# Graficar usando un mapa de colores que varía según el área
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(column='area_norm', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')

# Eliminar los ejes para una visualización más limpia
ax.set_axis_off()

# Mostrar el mapa
plt.show()


NameError: name 'gpd' is not defined

#### 2.
Crea un área de 5km alrededor de cada escuela. Calcula el área total cubierta y saca el porcentaje de área del país cubierta por escuelas.
Tip: cuidado con las intersecciones y las unidades!

In [ ]:
# Paso 1: Cargar el Shapefile de las escuelas
escuelas_gdf = gpd.read_file("./Escuelas/conjunto_de_datos/denue_inegi_61_.shp")

# Paso 2: Crear un buffer de 5 km alrededor de cada escuela
# Asegúrate de que el CRS (Sistema de Referencia de Coordenadas) sea adecuado para medidas de distancia en metros
escuelas_gdf = escuelas_gdf.to_crs(epsg=6362)  # Cambia el EPSG según tu zona
escuelas_gdf['buffer'] = escuelas_gdf['geometry'].buffer(5000)  # 5000 metros = 5 km

# Paso 3: Fusionar buffers superpuestos usando la operación de disolución
buffers_union = escuelas_gdf['buffer'].unary_union

# Paso 4: Calcular el área total ocupada por los buffers
total_area_escuelas = buffers_union.area  # El área estará en metros cuadrados si el CRS está en metros

# Paso 5: Comparar con el área total del país
# Paso 1: Cargar el Shapefile de los estados
estados_gdf = gpd.read_file("Estados/conjunto_de_datos/00ent.shp")

# Paso 2: Asegurarse de que el CRS sea adecuado para medidas de distancia en metros
# El CRS dependerá de los datos; aquí se usa un ejemplo genérico.
estados_gdf = estados_gdf.to_crs(epsg=6362)  # Cambia el EPSG según tu zona

# Calcular el área de cada estado
estados_gdf['area'] = estados_gdf['geometry'].area

# Paso 3: Sumar las áreas de todos los estados para obtener el área total del país
area_total_pais = estados_gdf['area'].sum()

# Imprimir resultados
print(f"Área total ocupada por las escuelas (con buffers de 5 km): {total_area_escuelas} m²")
print(f"Área total del país: {area_total_pais} m²")
print(f"Porcentaje del área ocupada por escuelas: {(total_area_escuelas / area_total_pais) * 100:.4f}%")


#### 3.
Calcula la distancia entre la CDMX y todos los demás estados de la república. 
Bonus: punto extra si lo haces usando las fronteras y no los centroides.

### Nota importante 

Función distance de Shapely
La función distance calcula la distancia euclidiana mínima entre los bordes de dos objetos geométricos. En el contexto de los estados y la Ciudad de México, hace lo siguiente:

Argumentos: Toma dos objetos de tipo Shapely.geometry. En este caso, un objeto es la geometría de un estado y el otro es la geometría de la Ciudad de México.

Proceso: Encuentra los puntos más cercanos entre las dos geometrías proporcionadas, es decir, los puntos que tienen la menor distancia euclidiana entre ellos. Estos puntos pueden estar en el borde o en algún punto interno de las geometrías, dependiendo de sus formas.

Resultado: Devuelve la distancia en las unidades del sistema de referencia de coordenadas (CRS) actual del GeoDataFrame. Si el CRS está configurado para medir en metros, la distancia será en metros.

In [ ]:
#Primero ver donde esta el nombre de los estados
print(estados_gdf.columns)
# Paso 2: Identificar la geometría de la Ciudad de México
cdmx_geometry = estados_gdf.loc[estados_gdf['NOMGEO'] == 'Ciudad de México', 'geometry'].squeeze()

# Paso 3: Calcular la distancia más corta de cada estado a la Ciudad de México
estados_gdf['distancia_a_cdmx'] = estados_gdf['geometry'].apply(lambda x: x.distance(cdmx_geometry))

# Imprimir las distancias
for estado, distancia in zip(estados_gdf['NOMGEO'], estados_gdf['distancia_a_cdmx']):
    print(f"Distancia de {estado} a Ciudad de México: {distancia:.2f} metros")


#### 4.
Metete al INEGI y descarga el shapefile de nombre "Servicios profesionales, científicos y técnicos" de 2023 (más info en el READ ME y en la presentación). Grafica las escuelas (sólo las primarias, secundarias y prepas), y los servicios que se muestran en este shapefile (las escuelas de un color y los servicios de otro). También grafica los estados. Comenta si ves alguna relación entre la localización de estas escuelas y los servicios.

In [ ]:
servicios = gpd.read_file("/usr/src/app/Servicios/conjunto_de_datos/denue_inegi_54_.shp")

In [ ]:
servicios = gpd.read_file("/usr/src/app/Servicios/conjunto_de_datos/denue_inegi_54_.shp")

print(servicios.head())

In [ ]:
servicios = servicios.to_crs(estados_gdf.crs)

In [ ]:
# Regex para clasificar las escuelas
regex_escuelas = r"(?i)(primaria|escuela básica|escuela elemental|secundaria|escuela media|educación media|preparatoria|bachillerato|prepa|colegio)"

# Filtrar las escuelas que cumplen con el regex
escuelas_gdf_filtrado = escuelas_gdf[escuelas_gdf['nom_estab'].str.contains(regex_escuelas, na=False)]

# Configurar la figura y el eje para el plot
fig, ax = plt.subplots(figsize=(10, 10))

# Dibujar el mapa de los estados
estados_gdf.plot(ax=ax, color='white', edgecolor='black')

# Dibujar las escuelas filtradas
escuelas_gdf_filtrado.plot(ax=ax, color='limegreen', markersize=10, label='Escuelas')

# Dibujar los servicios
servicios.plot(ax=ax, color='red', markersize=10, label='Servicios')

# Configurar leyenda y otros elementos del gráfico
plt.legend(title='Categorías')
plt.title('Distribución de Escuelas y Servicios en México')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.grid(True)

# Mostrar el mapa
plt.show()

# Ver las primeras filas del DataFrame filtrado para confirmar
print(escuelas_gdf_filtrado.head())

# Contar cuántas filas cumplen con el criterio
print(f"Número de filas que cumplen con el criterio de escuelas: {len(escuelas_gdf_filtrado)}")

### Para Carla:

Como podemos ver el Norte no tiene nada que hacer contra la capital del mundo Puebla. Obvservamos que el centro es donde se concentra la mayor cantidad de ambos. En resumen, Puebla es el wakanda de Mexico